In [1]:
# import asyncio
%load_ext autoreload
%autoreload 2

from dotenv import dotenv_values
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.legacy.llms import OpenAILike as OpenAI
from qdrant_client import models
from tqdm.asyncio import tqdm

from pipeline.ingestion import build_pipeline, build_vector_store, read_data
from pipeline.qa import read_jsonl, save_answers
from pipeline.rag import QdrantRetriever, generation_with_knowledge_retrieval


config = dotenv_values(".env")
# config = {'COLLECTION_NAME': 'aiops24_large',
#           'VECTOR_SIZE': 768,
#           'GLM_KEY': 'ba7d0d7930f621b8f9d2036deb55207a.En856p5uJTTXbxQv'
# }
config


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OrderedDict([('COLLECTION_NAME', 'aiops24'),
             ('VECTOR_SIZE', '512'),
             ('GLM_KEY', 'ba7d0d7930f621b8f9d2036deb55207a.En856p5uJTTXbxQv')])

In [3]:

# 初始化 LLM 嵌入模型 和 Reranker
llm = OpenAI(
    api_key=config["GLM_KEY"],
    model="glm-4",
    api_base="https://open.bigmodel.cn/api/paas/v4/",
    is_chat_model=True,
)
embeding = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-zh-v1.5",
    # model_name="BAAI/bge-big-zh-v1.5",
    # model_name="BAAI/bge-large-zh-v1.5",
    cache_folder="./",
    embed_batch_size=128,
)
Settings.embed_model = embeding

# 初始化 数据ingestion pipeline 和 vector store
client, vector_store = await build_vector_store(config, reindex=False)

collection_info = await client.get_collection(
    config["COLLECTION_NAME"] or "aiops24"
)

print(collection_info.points_count)

0


In [2]:
print(collection_info.points_count)

0


In [7]:

if collection_info.points_count == 0:
    data = read_data("data")
    pipeline = build_pipeline(llm, embeding, vector_store=vector_store)
    # 暂时停止实时索引
    await client.update_collection(
        collection_name=config["COLLECTION_NAME"] or "aiops24",
        optimizer_config=models.OptimizersConfigDiff(indexing_threshold=0),
    )
    await pipeline.arun(documents=data, show_progress=True, num_workers=1)
    # 恢复实时索引
    await client.update_collection(
        collection_name=config["COLLECTION_NAME"] or "aiops24",
        optimizer_config=models.OptimizersConfigDiff(indexing_threshold=20000),
    )
    print(len(data))


Generating embeddings: 100%|██████████| 244/244 [04:12<00:00,  1.03s/it]  


42139


In [8]:

retriever = QdrantRetriever(vector_store, embeding, similarity_top_k=3)

queries = read_jsonl("question.jsonl")

print(len(queries))

103


In [5]:
retriever

In [74]:
from llama_index.core import QueryBundle
    

def find_cate(node):
    cates = set()
    for k, v in node.relationships.items():
        if 'file_path' in v.metadata:
            fp = v.metadata['file_path']
            cate = fp.replace('/mnt/workspace/aiops24-RAG-demo/demo/data/', '').split('/')[0]
            # print(cate)
            cates.add(cate)
    # print(cates)
    return list(cates)

hit = 0
nsame = 0
for query in queries:
    
    qd = query['document']
    query_str = query['query']
    query_bundle = QueryBundle(query_str=query_str)
    node_with_scores = await retriever.aretrieve(query_bundle)
    rds = set()
    for n in node_with_scores[0:1]:
        ds = find_cate(n.node)
        for d in ds:
            rds.add(d)
    
    if qd in rds:
        hit += 1
    
    if qd != ','.join(rds):
        print(query, rds)
        nsame += 1
    
hit, nsame


{'id': 2, 'query': 'ZXUN RCP部署成功后，各个虚机个数都是最少个数，是否可以一次性扩容完成？有哪些注意事项？', 'document': 'rcp'} {'director'}
{'id': 3, 'query': '如何排查PCF侧建立专载失败，发起Rx-ASR释放问题？', 'document': 'rcp'} {'umac'}
{'id': 8, 'query': 'RCP在IaaS架构下有哪些GSU虚机？', 'document': 'rcp'} {'umac'}
{'id': 9, 'query': 'RCP如何将VoNR呼叫的Rx会话绑定到对应的N7会话？', 'document': 'rcp'} {'umac'}
{'id': 14, 'query': 'EPS回落流程中，PCF向SBC上报几次用户位置吗，分别使用什么事件？', 'document': 'rcp'} {'umac'}
{'id': 21, 'query': 'RCP怎样实现语音会话隔离？', 'document': 'rcp'} {'umac'}
{'id': 23, 'query': 'RCP和SPR之间有哪些接口？分别实现什么功能？', 'document': 'rcp'} {'umac'}
{'id': 25, 'query': 'Director性能数据最长可以保存多久？', 'document': 'director'} {'rcp'}
{'id': 26, 'query': 'PCF作为服务端时，AMF和SMF一般根据什么发现PCF', 'document': 'rcp'} {'umac'}
{'id': 28, 'query': 'PCF与SMF对接时，一般需要配置哪些数据？', 'document': 'rcp'} {'umac'}
{'id': 30, 'query': 'Daisyseed安装软件从哪里获取', 'document': 'director'} {'umac'}
{'id': 35, 'query': '如何选择合适的信令跟踪', 'document': 'rcp'} {'umac'}
{'id': 36, 'query': '外部系统想要采集Director的性能数据，可以怎么解决？', 'document': 'direc

(74, 29)

In [77]:
retriever = QdrantRetriever(vector_store, embeding, similarity_top_k=3)

queries = read_jsonl("question.jsonl")

print(len(queries))

from custom.template import QA_TEMPLATE, QA_TEMPLATE_2

print(QA_TEMPLATE_2)

qid = [18]
# qid = [8, 9, 14, 17, 18, 20, 21, 23, 37, 72, 74, 92]


# 生成答案
print("Start generating answers...")

results = []
for query in queries:
    if query['id'] not in qid:
        continue
    print(query)
    result = await generation_with_knowledge_retrieval(
        query["query"], retriever, llm, qa_template=QA_TEMPLATE_2, 
        debug=True,
        only_retrieval=True,
    )
    results.append(result)
    


# 处理结果
# save_answers(queries, results, "submit_result.jsonl")


103
    上下文信息如下：
    ----------
    {context_str}
    ----------
    请你首选基于上下文信息而不是自己的知识，回答以下问题，可以分点作答，如果上下文信息没有相关知识，但你确实知道比较准确的答案，可以基于自己的知识进行回答，不要解释上下文中是否有，也不要复述上下文信息，尽量直接回答问题：
    {query_str}

    回答：    
Start generating answers...
{'id': 18, 'query': 'RCP包含哪些数据存储类服务？', 'document': 'rcp'}
retrieved:
[rcp]:# 数据服务通信配置
[rcp]:# 数据服务通信配置
[umac]:特性描述
[NodeWithScore(node=TextNode(id_='1b6d3612-b557-4988-92a8-f2d7f4bf6b6f', embedding=None, metadata={'file_path': 'Ncudr_SystemManagement/zh-cn/tree/1.txt', 'file_name': '1.txt', 'file_type': 'text/plain', 'file_size': 889, 'creation_date': '2024-06-15', 'last_modified_date': '2024-03-12', 'document_title': '# 数据服务通信配置'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_

In [76]:
for r in results:
    print(r.text)
    print('-------------')
    

RCP包含的数据存储类服务有UDR（User Data Repository，用户数据仓库）和UDSF（User Data Storage Function，用户数据存储功能）。其中UDR用于存储结构化数据，而UDSF用于存储非结构化数据。
-------------


In [87]:

# 生成答案
print("Start generating answers...")

results = []
for query in tqdm(queries, total=len(queries)):
    try:
        result = await generation_with_knowledge_retrieval(
            query["query"], retriever, llm, qa_template=QA_TEMPLATE_2,
        )
    except:
        result = None
        print(f'except {query}')
    results.append(result)

len(results)

# 处理结果
# save_answers(queries, results, "submit_result_small.jsonl")


Start generating answers...


100%|██████████| 103/103 [09:34<00:00,  5.57s/it]


103

In [89]:
import jsonlines

def read_jsonl(path):
    content = []
    with jsonlines.open(path, "r") as json_file:
        for obj in json_file.iter(type=dict, skip_invalid=True):
            content.append(obj)
    return content


def save_answers(
    queries, results, base_res, path: str = "data/answers.jsonl"
):
    answers = []
    for qi, (query, result) in enumerate(zip(queries, results)):
        if result is not None:
            answers.append(
                {"id": query["id"], "query": query["query"], "answer": result.text}
            )
        else:
            answers.append(
                {"id": query["id"], "query": query["query"], "answer": base_res[qi]['answer']}
            )
            print(f'{qi} {query} no res, use base res')
            print(base_res[qi]['answer'])

    # use jsonlines to save the answers
    def write_jsonl(path, content):
        with jsonlines.open(path, "w") as json_file:
            json_file.write_all(content)

    # 保存答案到 data/answers.jsonl
    write_jsonl(path, answers)

    
base_res = read_jsonl('./submit_result_1.jsonl')
print(len(base_res))


# 处理结果
save_answers(queries, results, base_res, "submit_result_small.jsonl")

103


In [86]:
base_res[0]['answer']

'PCF与NRF对接时，一般需要配置以下数据：\n\n1. NRF Client配置：\n   - 是否启用SBI-GW功能\n\n2. NRF服务器分组配置：\n   - NRF服务器组编号\n\n3. NRF服务器节点配置：\n   - NRF服务器节点编号\n   - NRF服务器IP地址\n   - NRF服务器端口\n   - URI scheme\n   - API版本\n   - HTTP客户端模板编号\n   - 通知时使用的HTTP服务端模板编号\n   - NRF服务器节点优先级\n   - 归属的NRF服务器组编号\n\n4. NRF服务器策略配置：\n   - NRF服务器策略编号\n   - NRF模式\n   - 主用恢复后启用方式\n   - 临时重定向次数\n   - 主备不可用响应码\n   - 主备间永久重定向响应码\n   - 是否启用流控功能\n   - 心跳间隔(秒)\n   - 负载上报变化量阈值(%)\n\n5. NRF服务器模板配置：\n   - NRF服务器模板编号\n   - NRF服务器策略编号\n   - 主用NRF服务器组编号\n   - 备用NRF服务器组编号\n   - 检测方法\n   - 检测使用的NF实例标识\n   - 异常不可用检测次数\n   - 可用检测次数\n   - 检测间隔（秒）\n   - 服务信息格式类型配置\n\n6. NRF服务器模板选择配置：\n   - NF类型\n   - NRF服务器模板编号\n\n7. 重选配置：\n   - 目的NF类型\n   - 链路重选次数\n   - IP重选次数\n   - NF重选次数\n   - 重选等待时长(秒)\n\n8. 订阅条件配置：\n   - ID\n   - 目的NF类型\n   - 条件类型\n   - 条件值\n   - 事件类型\n\n以上数据是基于上下文信息中提供的配置步骤和举例进行总结的。'

In [84]:
questions[0]

{'id': 1,
 'query': 'PCF与NRF对接时，一般需要配置哪些数据？',
 'answer': 'PCF与NRF对接时，一般需要配置以下数据：\n\n1. NRF Client配置：\n   - 是否启用SBI-GW功能\n\n2. NRF服务器分组配置：\n   - NRF服务器组编号\n\n3. NRF服务器节点配置：\n   - NRF服务器节点编号\n   - NRF服务器IP地址\n   - NRF服务器端口\n   - URI scheme\n   - API版本\n   - HTTP客户端模板编号\n   - 通知时使用的HTTP服务端模板编号\n   - NRF服务器节点优先级\n   - 归属的NRF服务器组编号\n\n4. NRF服务器策略配置：\n   - NRF服务器策略编号\n   - NRF模式\n   - 主用恢复后启用方式\n   - 临时重定向次数\n   - 主备不可用响应码\n   - 主备间永久重定向响应码\n   - 是否启用流控功能\n   - 心跳间隔(秒)\n   - 负载上报变化量阈值(%)\n\n5. NRF服务器模板配置：\n   - NRF服务器模板编号\n   - NRF服务器策略编号\n   - 主用NRF服务器组编号\n   - 备用NRF服务器组编号\n   - 检测方法\n   - 检测使用的NF实例标识\n   - 异常不可用检测次数\n   - 可用检测次数\n   - 检测间隔（秒）\n   - 服务信息格式类型配置\n\n6. NRF服务器模板选择配置：\n   - NF类型\n   - NRF服务器模板编号\n\n7. 重选配置：\n   - 目的NF类型\n   - 链路重选次数\n   - IP重选次数\n   - NF重选次数\n   - 重选等待时长(秒)\n\n8. 订阅条件配置：\n   - ID\n   - 目的NF类型\n   - 条件类型\n   - 条件值\n   - 事件类型\n\n以上数据是基于上下文信息中提供的配置步骤和举例进行总结的。'}

In [82]:
import jsonlines

def read_jsonl(path):
    content = []
    with jsonlines.open(path, "r") as json_file:
        for obj in json_file.iter(type=dict, skip_invalid=True):
            content.append(obj)
    return content

questions = read_jsonl('./submit_result_1.jsonl')

for q in questions:
    print(q['answer'])
    
    break

PCF与NRF对接时，一般需要配置以下数据：

1. NRF Client配置：
   - 是否启用SBI-GW功能

2. NRF服务器分组配置：
   - NRF服务器组编号

3. NRF服务器节点配置：
   - NRF服务器节点编号
   - NRF服务器IP地址
   - NRF服务器端口
   - URI scheme
   - API版本
   - HTTP客户端模板编号
   - 通知时使用的HTTP服务端模板编号
   - NRF服务器节点优先级
   - 归属的NRF服务器组编号

4. NRF服务器策略配置：
   - NRF服务器策略编号
   - NRF模式
   - 主用恢复后启用方式
   - 临时重定向次数
   - 主备不可用响应码
   - 主备间永久重定向响应码
   - 是否启用流控功能
   - 心跳间隔(秒)
   - 负载上报变化量阈值(%)

5. NRF服务器模板配置：
   - NRF服务器模板编号
   - NRF服务器策略编号
   - 主用NRF服务器组编号
   - 备用NRF服务器组编号
   - 检测方法
   - 检测使用的NF实例标识
   - 异常不可用检测次数
   - 可用检测次数
   - 检测间隔（秒）
   - 服务信息格式类型配置

6. NRF服务器模板选择配置：
   - NF类型
   - NRF服务器模板编号

7. 重选配置：
   - 目的NF类型
   - 链路重选次数
   - IP重选次数
   - NF重选次数
   - 重选等待时长(秒)

8. 订阅条件配置：
   - ID
   - 目的NF类型
   - 条件类型
   - 条件值
   - 事件类型

以上数据是基于上下文信息中提供的配置步骤和举例进行总结的。
